In [1]:
%matplotlib inline
import networkx as nx
import pandas as pd

In [2]:
df = pd.read_json('data/movies-2000.json')
df

,productId,userId,profileName,helpfulness,score,time,summary,text,year,month,day
22,B00004CQT3,A1VJCDRXUQVXBM,K. Kangley,10/11,5,964915200000,A truly wonderful family film,I would beg to differ with the above descripti...,2000,7,30
418,0790747324,A96DHMTN072M1,James M. Cargal,1/1,5,968457600000,An Enduring Classic,By today's standards this movie might be weak....,2000,9,9
419,0790747324,A1L7G9JSLCY384,David Crosby,1/1,5,962409600000,The all TIME classic adventure,This film ranks as quite literaly my all time ...,2000,7,1
437,0790747324,A22IK3I6U76GX0,Calle,2/3,5,977875200000,"A great, very entertaining movie!",This movie is very entertaining. An inventor h...,2000,12,27
489,0790747324,A1BWHXTFOVF9CY,Blackhawk,60/62,5,970963200000,The Time Machine DVD Delivers,"Like most people, I had only seen this classic...",2000,10,8
...,...,...,...,...,...,...,...,...,...,...,...
7911290,6300181243,A27PSZX2SE0B51,Mike Stone,11/14,4,968544000000,Beware of the dreaded left-handed moth!,Once you get past the opening song and dance n...,2000,9,10
7911298,6300181243,A1TQB982TY31QV,plsilverman,3/4,3,960940800000,A Glimpse Into The Marxs' Past...and Future,Highly reminiscent of their first talkie &quot...,2000,6,14
7911299,6300181243,A1IOM171PPQ2U8,"""mr_nasty""",6/9,5,956793600000,"Hooray, Hooray, Hooray",Wild comedy from the Marx Brothers which is gu...,2000,4,27
7911430,6304026668,A3BP3VXMK6W0PR,"""lecter5000""",0/0,3,958348800000,Forgotten classic,This film is the ultimate in late night entert...,2000,5,15


In [3]:
df_relevant_cols = df[['productId', 'userId']]
df_relevant_cols

,productId,userId
22,B00004CQT3,A1VJCDRXUQVXBM
418,0790747324,A96DHMTN072M1
419,0790747324,A1L7G9JSLCY384
437,0790747324,A22IK3I6U76GX0
489,0790747324,A1BWHXTFOVF9CY
...,...,...
7911290,6300181243,A27PSZX2SE0B51
7911298,6300181243,A1TQB982TY31QV
7911299,6300181243,A1IOM171PPQ2U8
7911430,6304026668,A3BP3VXMK6W0PR


In [4]:
edgelist_df = df_relevant_cols.copy(deep=True).merge(df_relevant_cols,
                                                     how='inner',
                                                     on='productId',
                                                     suffixes=['_left', '_right'])
edgelist_df = edgelist_df.where(edgelist_df['userId_left'] < edgelist_df['userId_right']).dropna()
edgelist_df

,productId,userId_left,userId_right
6,0790747324,A96DHMTN072M1,AES0JMK0KYYHI
11,0790747324,A96DHMTN072M1,AXY18W5QMN7UV
12,0790747324,A1L7G9JSLCY384,A96DHMTN072M1
14,0790747324,A1L7G9JSLCY384,A22IK3I6U76GX0
17,0790747324,A1L7G9JSLCY384,AES0JMK0KYYHI
...,...,...,...
8916565,B000ZLRGCY,A3B249FQ2OJBCQ,A3DL1YXKDVESZH
8916566,B000ZLRGCY,A3B249FQ2OJBCQ,AT6JF8GKDMKS0
8916581,6300181243,A1TQB982TY31QV,A27PSZX2SE0B51
8916584,6300181243,A1IOM171PPQ2U8,A27PSZX2SE0B51


In [5]:
G: nx.Graph = nx.from_pandas_edgelist(edgelist_df,
                            source='userId_left',
                            target='userId_right',
                            edge_attr=['productId'])

In [6]:
G.number_of_nodes()

21214

In [7]:
G.number_of_edges()

333409

In [8]:
nx.write_gexf(G, 'generated_graphs/reviewed_same_movie.gexf')